In [13]:
import os
import uuid

from datetime import datetime, timedelta
import values, helpers
import random
import pandas as pd
import numpy as np
import create_sap_table.create_table_leanx as sap_table

In [14]:
# required tables
VBAK = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='VBAK')])
VBAP = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='VBAP')])

LIKP = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='LIKP')])
LIPS = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='LIPS')])

MKPF = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='MKPF')])
MSEG = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='MSEG')])

BKPF = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='BKPF')])
BSEG = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='BSEG')])

VBRK = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='VBRK')])
VBRP = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='VBRP')])

VBFA = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='VBFA')])

CDHDR = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='CDHDR')])
CDPOS = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='CDPOS')])

USR02 = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='USR02')])
MARA = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='MARA')])
KNA1 = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='KNA1')])

In [15]:
class User:
    def __init__(self, bname, ustyp, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.bname = bname
        self.ustyp = ustyp

class Material:
    def __init__(self, matnr, price, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.matnr = matnr
        self.price = price

class Customer:
    def __init__(self, kunnr, erdat, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.kunnr = kunnr
        self.erdat = erdat
        
class SalesOrderItem:
    def __init__(self, vbeln, posnr, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.vbeln = vbeln
        self.posnr = posnr

In [16]:
# insert tables and create objects list
USERS = []
for k, v in values.users.items():
    users_last_index = len(USR02)
    USR02.loc[users_last_index, 'MANDT'] = values.mandt
    USR02.loc[users_last_index, 'BNAME'] = k
    USR02.loc[users_last_index, 'USTYP'] = v

    USERS.append(User(bname=k, ustyp=v))

CUSTOMERS = []
for k, v in values.customers.items():
    kna1_last_index = len(KNA1)
    rand_kunnr = uuid.uuid4()
    rand_erdat = helpers.generate_random_datetime(start_date=datetime(2009, 1, 1), end_date=datetime(2010, 1, 1))

    KNA1.loc[kna1_last_index, 'MANDT'] = values.mandt
    KNA1.loc[kna1_last_index, 'KUNNR'] = rand_kunnr

    CUSTOMERS.append(Customer(kunnr=rand_kunnr, erdat=rand_erdat))

MATERIALS = []
for k, v in values.materials.items():
    mara_last_index = len(MATERIALS)
    rand_matnr = uuid.uuid4()

    MARA.loc[mara_last_index, 'MANDT'] = values.mandt
    MARA.loc[mara_last_index, 'MATNR'] = rand_matnr

    MATERIALS.append(Material(matnr=rand_matnr, price=v['price']))

In [17]:
class SalesOrder:
    def __init__(self, ernam, erdat, vbtyp, customer: Customer, agreed_delivery_time: datetime) -> None:
        self.mandt = values.mandt
        self.vbeln = uuid.uuid4()
        self.erdat = erdat
        self.ernam = ernam
        self.vbtyp = vbtyp
        self.netwr = 0
        self.customer = customer
        self.delco = agreed_delivery_time

        self.vbaps = []
        self.likp_id = uuid.uuid4()
        self.mkpf_mblnr = uuid.uuid4()
        self.vbrk_id = uuid.uuid4()
        self.bkpf_id = uuid.uuid4()

        self.activity_create_sales_order()

    def activity_create_sales_order(self):
        vbak_last_index = len(VBAK)
        VBAK.loc[vbak_last_index, 'MANDT'] = self.mandt
        VBAK.loc[vbak_last_index, 'VBELN'] = self.vbeln
        VBAK.loc[vbak_last_index, 'ERNAM'] = self.ernam
        VBAK.loc[vbak_last_index, 'ERDAT'] = self.erdat
        VBAK.loc[vbak_last_index, 'VBTYP'] = self.vbtyp
        VBAK.loc[vbak_last_index, 'KUNNR'] = self.customer
    
    def activity_create_sales_order_item(self, materials: list):
        for i, mat in enumerate(materials):
            quantity = random.randint(25, 150)
            vbap_last_index = len(VBAP)

            VBAP.loc[vbap_last_index, 'MANDT'] = self.mandt
            VBAP.loc[vbap_last_index, 'VBELN'] = self.vbeln
            VBAP.loc[vbap_last_index, 'POSNR'] = i+1
            VBAP.loc[vbap_last_index, 'MATNR'] = mat.matnr
            VBAP.loc[vbap_last_index, 'KWMENG'] = quantity # Cumulative Order Quantity in Sales Units
            VBAP.loc[vbap_last_index, 'NETWR'] = mat.price * 12 * 12 * quantity # calculated as a box which usually contains 12 dozens
            VBAP.loc[vbap_last_index, 'VBELN'] = self.vbeln

            self.netwr += mat.price * 12 * 12 * quantity
            self.vbaps.append(SalesOrderItem(vbeln=self.vbeln, posnr=i))

    def activity_generate_delivery_doc(self, delivery_doc_created_at: datetime, activity_by: User):        
        # generate the delivery document
        likp_last_index = len(LIKP)
        
        LIKP.loc[likp_last_index, 'MANDT'] = self.mandt
        LIKP.loc[likp_last_index, 'VBELN'] = self.likp_id
        LIKP.loc[likp_last_index, 'ERDAT'] = delivery_doc_created_at
        LIKP.loc[likp_last_index, 'ERNAM'] = activity_by

        # record LIPS
        for vbap in self.vbaps:
            lips_last_index = len(LIPS)

            LIPS.loc[lips_last_index, 'MANDT'] = self.mandt
            LIPS.loc[lips_last_index, 'VBELN'] = self.likp_id
            LIPS.loc[lips_last_index, 'POSNR'] = vbap.posnr

            # record the document flow
            vbfa_last_index = len(VBFA)
            VBFA.loc[vbfa_last_index, 'MANDT'] = self.mandt
            VBFA.loc[vbfa_last_index, 'ERDAT'] = delivery_doc_created_at
            VBFA.loc[vbfa_last_index, 'VBELV'] = vbap.vbeln
            VBFA.loc[vbfa_last_index, 'POSNV'] = vbap.posnr
            VBFA.loc[vbfa_last_index, 'VBTYP_V'] = 'C' # Order
            VBFA.loc[vbfa_last_index, 'VBELN'] = self.likp_id
            VBFA.loc[vbfa_last_index, 'POSNN'] = vbap.posnr # FIXME for cases where there is no one to one mapping of item positions
            VBFA.loc[vbfa_last_index, 'VBTYP_N'] = 'J'

    def activity_release_delivery(self, delivery_released_at: datetime, activity_by: User):
        # https://leanx.eu/en/sap/table/lips.html
        rand_change_nr = uuid.uuid4()
        
        # TODO include VBLB table for release type in conneciton with VBAK, VBAP, VBUK, VBUP
        # record LIPS
        for lips_index in LIPS[LIPS['VBELN'] == self.likp_id].index.values:
            cdpos_last_index = len(CDPOS)
            value_old = LIPS.loc[lips_index, 'ABART']
            LIPS.loc[lips_index, 'ABART'] = 6

            # record change CDPOS
            CDPOS.loc[cdpos_last_index, 'MANDANT'] = self.mandt
            CDPOS.loc[cdpos_last_index, 'OBJECTCLAS'] = "LIPS"
            CDPOS.loc[cdpos_last_index, 'OBJECTID'] = f"{self.mandt}{LIPS.loc[lips_index, 'VBELN']}{LIPS.loc[lips_index, 'POSNR']}"
            CDPOS.loc[cdpos_last_index, 'CHANGENR'] = rand_change_nr
            CDPOS.loc[cdpos_last_index, 'TABNAME'] = "LIPS"
            CDPOS.loc[cdpos_last_index, 'TABKEY'] = f"{self.mandt}{LIPS.loc[lips_index, 'VBELN']}"
            CDPOS.loc[cdpos_last_index, 'FNAME'] = 'ABART'
            CDPOS.loc[cdpos_last_index, 'CHNGIND'] ='U'
            CDPOS.loc[cdpos_last_index, 'VALUE_OLD'] = value_old
            CDPOS.loc[cdpos_last_index, 'VALUE_NEW'] = LIPS.loc[lips_index, 'ABART']

        # record change CDHDR
        cdhdr_last_index = len(CDHDR)
        CDHDR.loc[cdhdr_last_index, 'MANDANT'] = self.mandt
        CDHDR.loc[cdhdr_last_index, 'OBJECTCLAS'] = "LIPS"
        CDHDR.loc[cdhdr_last_index, 'OBJECTID'] = self.likp_id
        CDHDR.loc[cdhdr_last_index, 'CHANGENR'] = rand_change_nr
        CDHDR.loc[cdhdr_last_index, 'USERNAME'] = activity_by
        CDHDR.loc[cdhdr_last_index, 'UDATE'] = delivery_released_at

    def activity_ship_goods(self, shipped_at: datetime, activity_by: User):
        # record new MKPF
        mkpf_last_index = len(MKPF)

        MKPF.loc[mkpf_last_index, 'MANDT'] = self.mandt
        MKPF.loc[mkpf_last_index, 'MBLNR'] = self.mkpf_mblnr
        MKPF.loc[mkpf_last_index, 'SPE_BUDAT_UHR'] = shipped_at
        MKPF.loc[mkpf_last_index, 'USNAM'] = activity_by

        # record new MSEG
        for vbap in self.vbaps:
            mseg_last_index = len(MSEG)

            MSEG.loc[mseg_last_index, 'MANDT'] = self.mandt
            MSEG.loc[mseg_last_index, 'MBLNR'] = self.mkpf_mblnr
            MSEG.loc[mseg_last_index, 'ZEILE'] = vbap.posnr
            MSEG.loc[mseg_last_index, 'KDAUF'] = self.vbeln
            MSEG.loc[mseg_last_index, 'KDPOS'] = vbap.posnr

        # record the document flow
        for lips_index in LIPS[LIPS['VBELN'] == self.likp_id].index.values:
            vbfa_last_index = len(VBFA)
            VBFA.loc[vbfa_last_index, 'MANDT'] = self.mandt
            VBFA.loc[vbfa_last_index, 'ERDAT'] = shipped_at
            VBFA.loc[vbfa_last_index, 'VBELV'] = LIPS.loc[lips_index, 'VBELN']
            VBFA.loc[vbfa_last_index, 'POSNV'] = LIPS.loc[lips_index, 'POSNR'] 
            VBFA.loc[vbfa_last_index, 'ERDAT'] = shipped_at
            VBFA.loc[vbfa_last_index, 'VBTYP_V'] = 'J' # Delivery
            VBFA.loc[vbfa_last_index, 'VBELN'] = self.mkpf_mblnr
            VBFA.loc[vbfa_last_index, 'POSNN'] = LIPS.loc[lips_index, 'POSNR'] # TODO change to MSEG ZEILE  # FIXME for cases where there is no one to one mapping of item positions
            VBFA.loc[vbfa_last_index, 'VBTYP_N'] = 'R' # Goods movement
    
    def activity_create_billing_document(self, invoice_sent_at: datetime, activity_by: User):
        # record new billing document (VBRK)
        vbrk_last_index = len(VBRK)
        VBRK.loc[vbrk_last_index, 'MANDT'] = self.mandt
        VBRK.loc[vbrk_last_index, 'VBELN'] = self.vbrk_id
        VBRK.loc[vbrk_last_index, 'ERNAM'] = activity_by
        VBRK.loc[vbrk_last_index, 'ERDAT'] = invoice_sent_at

        # create VBRP
        for vbap in self.vbaps:
            vbrp_last_index = len(VBRP)
            VBRP.loc[vbrp_last_index, 'MANDT'] = self.mandt
            VBRP.loc[vbrp_last_index, 'VBELN'] = self.vbrk_id
            VBRP.loc[vbrp_last_index, 'POSNR'] = vbap.posnr

        # record VBFA
        for mseg_index in MSEG[MSEG['MBLNR'] == self.mkpf_mblnr].index.values:
            vbfa_last_index = len(VBFA)
            VBFA.loc[vbfa_last_index, 'MANDT'] = self.mandt
            VBFA.loc[vbfa_last_index, 'ERDAT'] = invoice_sent_at
            VBFA.loc[vbfa_last_index, 'VBELV'] = MSEG.loc[mseg_index, 'MBLNR']
            VBFA.loc[vbfa_last_index, 'POSNV'] = MSEG.loc[mseg_index, 'ZEILE']
            VBFA.loc[vbfa_last_index, 'VBTYP_V'] = 'R' # Goods movement
            VBFA.loc[vbfa_last_index, 'VBELN'] = self.vbrk_id
            VBFA.loc[vbfa_last_index, 'POSNN'] = MSEG.loc[mseg_index, 'ZEILE'] # TODO change to VBRP POSNR  # FIXME for cases where there is no one to one mapping of item positions
            VBFA.loc[vbfa_last_index, 'VBTYP_N'] = 'M' # Invoice
            VBFA.loc[vbfa_last_index, 'ERDAT'] = invoice_sent_at

        # record new invoice (BKPF)  ---Create Invoice---
        bkpf_last_index = len(BKPF)
        BKPF.loc[bkpf_last_index, 'MANDT'] = self.mandt
        BKPF.loc[bkpf_last_index, 'BUKRS'] = self.customer
        BKPF.loc[bkpf_last_index, 'BELNR'] = self.bkpf_id
        BKPF.loc[bkpf_last_index, 'BLDAT'] = invoice_sent_at
        BKPF.loc[bkpf_last_index, 'USNAM'] = activity_by
        BKPF.loc[bkpf_last_index, 'AWKEY'] = self.vbrk_id
        BKPF.loc[bkpf_last_index, 'AWTYP'] = 'VBRK'

        # record invoice item (BSEG)
        for vbap in self.vbaps:
            bseg_last_index = len(BSEG)
            BSEG.loc[bseg_last_index, 'MANDT'] = self.mandt
            BSEG.loc[bseg_last_index, 'BUKRS'] = self.customer
            BSEG.loc[bseg_last_index, 'BELNR'] = self.bkpf_id
            BSEG.loc[bseg_last_index, 'VBEL2'] = self.vbeln
            BSEG.loc[bseg_last_index, 'POSN2'] = vbap.posnr
            BSEG.loc[bseg_last_index, 'BUZEI'] = vbap.posnr

    def activity_receive_delivery_confirmation(self, actual_delivery_at: datetime, delivery_confirmation_received_at: datetime, activity_by: User):
        # update delivery document header
        likp_index = LIKP[LIKP['VBELN'] == self.likp_id].index.values[0]
        
        value_old = LIKP.loc[likp_index, 'SPE_ACC_APP_STS']
        LIKP.loc[likp_index, 'SPE_ACC_APP_STS'] = 'C'
        LIKP.loc[likp_index, 'LFUHR'] = actual_delivery_at

        rand_change_nr = uuid.uuid4()
        cdpos_last_index = len(CDPOS)

        # record change CDPOS
        CDPOS.loc[cdpos_last_index, 'MANDANT'] = self.mandt
        CDPOS.loc[cdpos_last_index, 'OBJECTCLAS'] = "LIKP"
        CDPOS.loc[cdpos_last_index, 'OBJECTID'] = f"{self.mandt}{self.likp_id}"
        CDPOS.loc[cdpos_last_index, 'CHANGENR'] = rand_change_nr
        CDPOS.loc[cdpos_last_index, 'TABNAME'] = "LIKP"
        CDPOS.loc[cdpos_last_index, 'TABKEY'] = f"{self.mandt}{self.likp_id}"
        CDPOS.loc[cdpos_last_index, 'FNAME'] = 'SPE_ACC_APP_STS'
        CDPOS.loc[cdpos_last_index, 'CHNGIND'] ='U'
        CDPOS.loc[cdpos_last_index, 'VALUE_OLD'] = value_old
        CDPOS.loc[cdpos_last_index, 'VALUE_NEW'] = LIKP.loc[likp_index, 'SPE_ACC_APP_STS']

        # record change CDHDR
        cdhdr_last_index = len(CDHDR)
        CDHDR.loc[cdhdr_last_index, 'MANDANT'] = self.mandt
        CDHDR.loc[cdhdr_last_index, 'OBJECTCLAS'] = "LIKP"
        CDHDR.loc[cdhdr_last_index, 'OBJECTID'] = self.likp_id
        CDHDR.loc[cdhdr_last_index, 'CHANGENR'] = rand_change_nr
        CDHDR.loc[cdhdr_last_index, 'USERNAME'] = activity_by
        CDHDR.loc[cdhdr_last_index, 'UDATE'] = delivery_confirmation_received_at

    def activity_clear_invoice(self, inovice_cleared_at: datetime):
        bseg_index = BSEG[BSEG['BELNR'] == self.bkpf_id].index.values[0]
        BSEG.loc[bseg_index, 'AUGDT'] = inovice_cleared_at

In [20]:
so_creator = USERS[0].bname
latest_time = helpers.generate_random_datetime(start_date=datetime(2009, 1, 1), end_date=datetime(2010, 1, 1))
shipping_condition = 1
customer = CUSTOMERS[0].kunnr
agreed_delivery_time = latest_time + helpers.UPTO_YEAR()

sales_order = SalesOrder(ernam=so_creator, erdat=latest_time, vbtyp=shipping_condition, customer=customer, agreed_delivery_time=agreed_delivery_time)

materials = MATERIALS[-3:]
sales_order.activity_create_sales_order_item(materials=materials)

delivery_doc_created_by = USERS[0].bname
latest_time += helpers.UPTO_WEEK()
sales_order.activity_generate_delivery_doc(delivery_doc_created_at=latest_time, activity_by=delivery_doc_created_by)

delivery_released_by = USERS[0].bname
latest_time += helpers.UPTO_WEEK()
sales_order.activity_release_delivery(delivery_released_at=latest_time, activity_by=delivery_released_by)

shipped_by = USERS[0].bname
latest_time += helpers.UPTO_WEEK()
sales_order.activity_ship_goods(shipped_at=latest_time, activity_by=shipped_by)

invoice_sent_by = USERS[0].bname
latest_time += helpers.UPTO_WEEK()
sales_order.activity_create_billing_document(invoice_sent_at=latest_time, activity_by=invoice_sent_by)

delivery_received_by = USERS[0].bname
latest_time += helpers.UPTO_WEEK()
sales_order.activity_receive_delivery_confirmation(actual_delivery_at=latest_time, delivery_confirmation_received_at=latest_time, activity_by=delivery_received_by)

invoice_cleared_by = USERS[0].bname
latest_time += helpers.UPTO_WEEK()
sales_order.activity_clear_invoice(inovice_cleared_at=latest_time)

In [21]:
version = str(datetime.now())
os.mkdir(f'data/{version}')

VBAK.to_csv(f'data/{version}/VBAK.csv', index=False)
VBAP.to_csv(f'data/{version}/VBAP.csv', index=False)

LIKP.to_csv(f'data/{version}/LIKP.csv', index=False)
LIPS.to_csv(f'data/{version}/LIPS.csv', index=False)

MKPF.to_csv(f'data/{version}/MKPF.csv', index=False)
MSEG.to_csv(f'data/{version}/MSEG.csv', index=False)

BKPF.to_csv(f'data/{version}/BKPF.csv', index=False)
BSEG.to_csv(f'data/{version}/BSEG.csv', index=False)

VBRK.to_csv(f'data/{version}/VBRK.csv', index=False)
VBRP.to_csv(f'data/{version}/VBRP.csv', index=False)

VBFA.to_csv(f'data/{version}/VBFA.csv', index=False)

CDHDR.to_csv(f'data/{version}/CDHDR.csv', index=False)
CDPOS.to_csv(f'data/{version}/CDPOS.csv', index=False)

USR02.to_csv(f'data/{version}/USR02.csv', index=False)
MARA.to_csv(f'data/{version}/MARA.csv', index=False)
KNA1.to_csv(f'data/{version}/KNA1.csv', index=False)

In [22]:
VBAP

,MANDT,VBELN,POSNR,MATNR,MATWA,PMATN,CHARG,MATKL,ARKTX,PSTYV,...,CANCEL_ALLOW,PAY_METHOD,BPN,REP_FREQ,PARGB,AUFPL_OAA,APLZL_OAA,ARSNUM,ARSPOS,WTYSC_CLMITEM
0,SC1,107338a3-630a-4db8-9e51-bcc6a614f281,1,20d38a5b-8ee6-41a2-a9cd-33ac20a5f04b,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SC1,107338a3-630a-4db8-9e51-bcc6a614f281,2,685d0761-f215-49fe-8c51-1f5078773d0c,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SC1,107338a3-630a-4db8-9e51-bcc6a614f281,3,af35c243-7820-406c-aa69-565dda093491,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SC1,d6fe9fbd-bcf1-4b1c-8efb-6af03b1ada87,1,20d38a5b-8ee6-41a2-a9cd-33ac20a5f04b,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SC1,d6fe9fbd-bcf1-4b1c-8efb-6af03b1ada87,2,685d0761-f215-49fe-8c51-1f5078773d0c,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SC1,d6fe9fbd-bcf1-4b1c-8efb-6af03b1ada87,3,af35c243-7820-406c-aa69-565dda093491,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SC1,194c40cb-183e-4747-b842-3c27dd7999d7,1,20d38a5b-8ee6-41a2-a9cd-33ac20a5f04b,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SC1,194c40cb-183e-4747-b842-3c27dd7999d7,2,685d0761-f215-49fe-8c51-1f5078773d0c,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SC1,194c40cb-183e-4747-b842-3c27dd7999d7,3,af35c243-7820-406c-aa69-565dda093491,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
class BillingDeviation:
    def __init__(self, vbak: SalesOrder) -> None:
        self.vbak = vbak
        
    def set_billing_block(self):
        # Activity: Set billing block
        self.vbak.faksk = 'BILLINGBLOCK'

        rand_changenr = uuid.uuid4()

        # assign user
        if random.uniform(0, 1) < 0.6: 
            user = 'BATCH_JOB'
        else:
            user = random.choice(list(values.users))

        # impact of user type
        if user == 'BATCH_JOB':
            billing_block_set_at =  self.vbak.latest_activity_at + helpers.UPTO_HOUR()
        else:
            billing_block_set_at =  self.vbak.latest_activity_at + helpers.UPTO_DAY()

        # increase cycle time
        self.vbak.update_latest_activity_time(new_latest_time=billing_block_set_at + helpers.UPTO_MONTH())

        self.vbak.cdhdr_changes_temp_list.append([values.mandt, 'BILLING', self.vbak.vbeln, rand_changenr, user, billing_block_set_at])
        self.vbak.cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBAK', 'FAKSK', pd.NA, 'BILLINGBLOCK'])
        
        self.vbak.has_billing_block = True

    def remove_billing_block(self):
        # Activity: Remove billing block
        self.vbak.faksk = pd.NA

        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users))

        billing_block_removed_at = self.vbak.latest_activity_at + helpers.UPTO_WEEK()
        self.vbak.update_latest_activity_time(new_latest_time=billing_block_removed_at)

        self.vbak.cdhdr_changes_temp_list.append([values.mandt, 'BILLING', self.vbak.vbeln, rand_changenr, rand_usr, billing_block_removed_at])
        self.vbak.cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBAK', 'FAKSK', 'BILLINGBLOCK', pd.NA])

class CancelOrReturn:
    def __init__(self, vbak) -> None:
        self.vbak = vbak
    
    def cancel_order(self):
        # Activity: Cancel order
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users))

        sd_cancelled_at = self.vbak.latest_activity_at + helpers.UPTO_DAY()
        self.vbak.update_latest_activity_time(new_latest_time=sd_cancelled_at)

        self.vbak.cdhdr_changes_temp_list.append([values.mandt, 'SALESORDER', self.vbak.vbeln, rand_changenr, rand_usr, sd_cancelled_at])
        self.vbak.cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'VBTYP', self.vbak.vbuk[-1], 'h'])
        self.vbak.vbuk[-1] = 'h'

    def return_goods(self):
        # Activity: Return goods
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users))

        sd_returned_at = self.vbak.latest_activity_at + helpers.UPTO_MONTH()
        self.vbak.update_latest_activity_time(new_latest_time=sd_returned_at)

        self.vbak.cdhdr_changes_temp_list.append([values.mandt, 'SALESORDER', self.vbak.vbeln, rand_changenr, rand_usr, sd_returned_at])
        self.vbak.cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'VBTYP', self.vbak.vbuk[-1], 'H'])
        self.vbak.vbuk[-1] = 'H'